In [16]:
!pip install git+https://github.com/huggingface/diffusers.git

  Cloning https://github.com/huggingface/diffusers.git to /tmp/pip-req-build-89cy6rgk
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers.git /tmp/pip-req-build-89cy6rgk
  Resolved https://github.com/huggingface/diffusers.git to commit 7ae6347e330a2b12da0b563370b6605a18bf0ed2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [17]:
!pip install transformers accelerate torch

In [18]:
!pip install flash-attn==2.7.3

In [19]:
import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.utils import export_to_video
from PIL import Image
import numpy as np

In [20]:
pipe = DiffusionPipeline.from_pretrained("cerspense/zeroscope_v2_576w", torch_dtype=torch.float16)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_vae_slicing()
pipe.to(device="cuda", dtype=torch.float16)
pipe.unet.enable_forward_chunking(chunk_size=1, dim=1)

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--cerspense--zeroscope_v2_576w/snapshots/6963642a64dbefa93663d1ecebb4ceda2d9ecb28/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--cerspense--zeroscope_v2_576w/snapshots/6963642a64dbefa93663d1ecebb4ceda2d9ecb28/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--cerspense--zeroscope_v2_576w/snapshots/6963642a64dbefa93663d1ecebb4ceda2d9ecb28/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--cerspense--zeroscope_v2_576w/snapshots/6963642a64dbefa93663d1ecebb4ceda2d9ecb28/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
The TextToVideoSDPipeline has been deprecated and will not receive bug fixes or feature updates after 

In [21]:
prompt = "Superman traveling in space"
video_frames = pipe(prompt, num_inference_steps=30, height=320, width=576, num_frames=25).frames

  0%|          | 0/30 [00:00<?, ?it/s]

In [22]:
# Convert the tensors to NumPy arrays and then to PIL Images
video_frames_pil = [Image.fromarray((frame * 255).astype(np.uint8)) for frame in video_frames[0]]
video_path = export_to_video(video_frames_pil, output_video_path="first.mp4")

In [24]:
from google.colab import files
from IPython.display import HTML
import base64




# Get the video file name
video_path = 'first.mp4'

# Convert video to base64 for inline playback
mp4 = open(video_path, 'rb').read()
data_url = "data:video/mp4;base64," + base64.b64encode(mp4).decode()

# Display the video
HTML(f"""
<video width=640 controls>
  <source src="{data_url}" type="video/mp4">
</video>
""")